## Test Data Preperation

Training and validation data went through multiple stages of data cleaning and feature engineering. The machine learning model is trained based on the features of train and valid data. Thus, we need to transform the test data in similar way to make prediction.

This notebook is dedicated to preparing the test data for making prediction.

In [1]:
### This cell contains all the tools required for the given task
import pandas as pd
import numpy as np

### Preporcessing tools
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce

In [2]:
### Loading the test data into dataframe using built-in pandas function
test_df = pd.read_csv('bluebook-for-bulldozers-data/Test.csv',low_memory=False,parse_dates=['saledate'])

In [3]:
test_df.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1227829,1006309,3168,121,3,1999,3688.0,Low,2012-05-03,580G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1227844,1022817,7271,121,3,1000,28555.0,High,2012-05-10,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1227847,1031560,22805,121,3,2004,6038.0,Medium,2012-05-10,EC210BLC,...,None or Unspecified,"9' 6""",Manual,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1227848,56204,1269,121,3,2006,8940.0,High,2012-05-10,330CL,...,None or Unspecified,None or Unspecified,Manual,Yes,Triple,NaN,NaN,NaN,NaN,NaN
4,1227863,1053887,22312,121,3,2005,2286.0,Low,2012-05-10,650K,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [4]:
### Converting the saledate to datetime format using to_datetime
test_df['saledate']= pd.to_datetime(test_df['saledate'])

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 0 to 12456
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SalesID                   12457 non-null  int64         
 1   MachineID                 12457 non-null  int64         
 2   ModelID                   12457 non-null  int64         
 3   datasource                12457 non-null  int64         
 4   auctioneerID              12457 non-null  int64         
 5   YearMade                  12457 non-null  int64         
 6   MachineHoursCurrentMeter  2129 non-null   float64       
 7   UsageBand                 1834 non-null   object        
 8   saledate                  12457 non-null  datetime64[ns]
 9   fiModelDesc               12457 non-null  object        
 10  fiBaseModel               12457 non-null  object        
 11  fiSecondaryDesc           8482 non-null   object        
 12  fiModelSeries     

In [6]:
### Changing 'None or Unspecified' feature value to None
for col in test_df.columns:
    test_df[col].mask(test_df[col] == 'None or Unspecified',None,inplace=True)

In [7]:
### We drop a lot of columns from the train-valid dataset because of large number of missing values
### We have to drop those columns from the test dataset as well because our ML model cannot utilise these features
### Using a list of column name to drop 
drop_col = ['UsageBand','fiModelSeries','fiModelDescriptor', 'Drive_System', 'Forks', 'Pad_Type', 'Ride_Control', 
            'Stick', 'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type','Engine_Horsepower',
            'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size','Coupler','Coupler_System',
            'Grouser_Tracks','Hydraulics_Flow','Track_Type','Undercarriage_Pad_Width','Stick_Length','Thumb',
            'Pattern_Changer','Grouser_Type','Backhoe_Mounting','Blade_Type','Travel_Controls','Differential_Type',
            'Steering_Controls','fiModelDesc','ProductGroupDesc']

for col in drop_col:
    test_df.drop(col,axis=1,inplace = True)

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 0 to 12456
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SalesID                   12457 non-null  int64         
 1   MachineID                 12457 non-null  int64         
 2   ModelID                   12457 non-null  int64         
 3   datasource                12457 non-null  int64         
 4   auctioneerID              12457 non-null  int64         
 5   YearMade                  12457 non-null  int64         
 6   MachineHoursCurrentMeter  2129 non-null   float64       
 7   saledate                  12457 non-null  datetime64[ns]
 8   fiBaseModel               12457 non-null  object        
 9   fiSecondaryDesc           8482 non-null   object        
 10  ProductSize               6048 non-null   object        
 11  fiProductClassDesc        12457 non-null  object        
 12  state             

In [9]:
### Filling up columns with the method used to fill the train-valid data

### using back-fill method for auctioneerId
test_df['auctioneerID'].bfill(axis=0,inplace=True)

### filling MachineHoursCurrentMeter with it's mean value 
test_df['MachineHoursCurrentMeter'].fillna(round(test_df['MachineHoursCurrentMeter'].mean(),2),inplace=True)

### filling enclosure column with absence value i.e. NO ROPS
test_df['Enclosure'].fillna('NO ROPS',inplace=True)

### replacing same variable with different text format
test_df['Transmission'].mask(test_df['Transmission'] == 'AutoShift','Autoshift',inplace = True)

### filling Transmission variable with 'Standard' 
test_df['Transmission'].fillna('Standard',inplace=True)

### using forward-fill method for auctioneerId
test_df['Hydraulics'].fillna(method='ffill',inplace=True)

### Imputing product size feature based on the product groups' high frequency value
test_df[test_df['ProductGroup']=='TTT']['ProductSize'].fillna('Medium',inplace = True)
test_df[test_df['ProductGroup']=='WL']['ProductSize'].fillna('Medium',inplace = True)
test_df[test_df['ProductGroup']=='TEX']['ProductSize'].fillna('Large / Medium',inplace = True)

### Imputing the remaining with the high frequency value
test_df['ProductSize'].fillna('Medium',inplace = True)

<ipython-input-9-2058d6e7a430>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[test_df['ProductGroup']=='TTT']['ProductSize'].fillna('Medium',inplace = True)
<ipython-input-9-2058d6e7a430>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[test_df['ProductGroup']=='WL']['ProductSize'].fillna('Medium',inplace = True)
<ipython-input-9-2058d6e7a430>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[test_df['Produc

In [10]:
## Define function for grouping state into categories
def bin_state(state):
    
    # check if U.S. territory or out of country
    if state in ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire','Rhode Island', 'Vermont',
                'New Jersey','New York', 'Pennsylvania']:
        return 'Northeast'
    
    elif state in ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia',
                    'Washington DC', 'West Virginia', 'Alabama', 'Kentucky', 'Mississippi',
                    'Tennessee','Arkansas', 'Louisiana', 'Oklahoma', 'Texas']:
        return 'South'
    
    elif state in ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa', 'Kansas', 
                   'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota']:
        return 'Midwest'
    
    elif state in ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah',
                   'Wyoming' , 'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']:
        return 'West'
    else:
        return 'Other'

In [11]:
### Creating the region column based on bin_State function
test_df['region'] = test_df['state'].apply(bin_state)

### Dropping the state column after creating the region column
test_df.drop('state',axis =1, inplace = True)

In [12]:
### Ordinal encoding order dictionary
encoding_order = {'ProductSize':[['Medium','Large','Large / Medium','Small','Mini','Compact']],
            'Enclosure':[['EROPS w AC','EROPS','OROPS','NO ROPS','EROPS AC']],
            'Transmission':[['Powershift','Standard','Autoshift','Hydrostatic','Direct Drive','Powershuttle']]}

In [13]:
### for loop to encode the value using the encoding_order dictionary
for key, value in encoding_order.items():
    o_enc = OrdinalEncoder(categories=value)
    test_df[key + '_Enc'] = o_enc.fit_transform(test_df[[key]])
    test_df.drop(key,axis = 1,inplace = True)

In [14]:
### Binary encoding of Hydraulics feature
bi_enc = ce.BinaryEncoder(test_df['Hydraulics'])
enc = bi_enc.fit_transform(test_df['Hydraulics'])
test_df = pd.concat([test_df,enc],axis = 1)
test_df.drop('Hydraulics',axis=1,inplace=True)

In [15]:
### Dummy Encoding
dummy_enc = pd.get_dummies(data = test_df[['region','ProductGroup']],
                           drop_first=True,
                           columns=['region','ProductGroup'],
                           dtype=float)

### Combining the dummy with the original dataframe
test_df = pd.concat([test_df,dummy_enc],axis = 1)

### Droping the original columns
test_df.drop(['region','ProductGroup'],axis=1,inplace = True)

In [16]:
### Turning string values into catagory using astype method.
for column, value in test_df.items():
    if test_df[column].dtypes == object:
        test_df[column] = value.astype('category')

In [17]:
### Check for labels that are catagorical
for column, value in test_df.items():
    if pd.api.types.is_categorical_dtype(value):
        ### Incrementing codes by 1 to fill the missing values
        test_df[column] =  pd.Categorical(value).codes + 1

In [18]:
test_df['YearMade'].value_counts().sort_index()

YearMade
1000    1637
1900       1
1957       1
1959       1
1961       4
1962       4
1963       1
1964       1
1965      12
1966      10
1967      14
1968       9
1969      20
1970      15
1971      13
1972      27
1973      21
1974      19
1975      16
1976      27
1977      38
1978      69
1979      48
1980      38
1981      30
1982      26
1983      31
1984      66
1985      75
1986      64
1987      99
1988      88
1989     136
1990     117
1991     101
1992     101
1993     121
1994     236
1995     254
1996     279
1997     351
1998     477
1999     486
2000     502
2001     422
2002     459
2003     525
2004     898
2005    1664
2006    1423
2007     792
2008     407
2009      83
2010      51
2011      34
2012      12
2014       1
Name: count, dtype: int64

In [19]:
test_df['YearMade'].mask(test_df['YearMade'] == 1000,1937,inplace=True)
test_df['YearMade'].mask(test_df['YearMade'] == 1900,1937,inplace=True)

In [20]:
## Add datetime parameter for saledate.
test_df['day'] = test_df.saledate.dt.day
test_df['month'] = test_df.saledate.dt.month
test_df['year'] = test_df.saledate.dt.year
test_df['DayOfWeek'] = test_df.saledate.dt.day_of_week
test_df['StartOfMonth'] = test_df.saledate.dt.is_month_start
test_df['EndOfYear'] = test_df.saledate.dt.is_year_end

test_df.drop('saledate',axis = 1,inplace = True)

In [21]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 0 to 12456
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   12457 non-null  int64  
 1   MachineID                 12457 non-null  int64  
 2   ModelID                   12457 non-null  int64  
 3   datasource                12457 non-null  int64  
 4   auctioneerID              12457 non-null  int64  
 5   YearMade                  12457 non-null  int64  
 6   MachineHoursCurrentMeter  12457 non-null  float64
 7   fiBaseModel               12457 non-null  int16  
 8   fiSecondaryDesc           12457 non-null  int8   
 9   fiProductClassDesc        12457 non-null  int8   
 10  ProductSize_Enc           12457 non-null  float64
 11  Enclosure_Enc             12457 non-null  float64
 12  Transmission_Enc          12457 non-null  float64
 13  Hydraulics_0              12457 non-null  int64  
 14  Hydrau

In [23]:
### Saving the processed test data into a json file for prediction
test_df.to_csv('Processed Data/Test_clean.csv',index=False)